In [ ]:
import pandas as pd
from typing import Dict

from test_data import create_sample_data, get_weekly_sales_data
from technical_interview_solution import process_sales_batch

Technical Interview Question 2:

2. For both `combine_sales_with_asset_data()` and `create_consolidated_weekly_report()` functions:
   - Explain the purpose of the function and its parameters to us.
   - Write a docstring and type hints for these functions.

In [ ]:
def combine_sales_with_asset_data(
    sales_results, 
    asset_data, 
):
    """
    Docstring
    """
    # Agg asset data by portfolio_id to get portfolio-level metrics
    asset_portfolio_summary = asset_data.groupby('portfolio_id').agg({
        'asset_id': 'count',
        'geography': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        'ISO': lambda x: list(x.unique()),
        'operational_date': ['min', 'max'],
        'timezone': lambda x: list(x.unique())
    })
    
    asset_portfolio_summary.columns = [
        'asset_count', 'primary_geography', 'iso_regions', 
        'oldest_asset_date', 'newest_asset_date', 'timezones'
    ]
    asset_portfolio_summary = asset_portfolio_summary.reset_index()
    
    # Calculate portfolio age in years. 365.25 is used to account for leap years.
    today = pd.Timestamp.now().date()
    asset_portfolio_summary['portfolio_age_years'] = asset_portfolio_summary['oldest_asset_date'].apply(
        lambda x: (today - x).days / 365.25
    ).round(1)
    
    combined_data = pd.merge(
        sales_results,
        asset_portfolio_summary,
        on='portfolio_id',
        how='left'
    )
    
    combined_data['mwh_per_asset'] = combined_data['MWh'] / combined_data['asset_count']
    combined_data['revenue_per_asset'] = combined_data['sales_amount'] / combined_data['asset_count']
    combined_data['revenue_per_mwh'] = combined_data['sales_amount'] / combined_data['MWh']

    
    return combined_data

In [ ]:
def create_consolidated_weekly_report(
        combined_weekly_data: Dict[str, pd.DataFrame]
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Creates a consolidated report by combining all batches of processed weekly data. 
    
    Parameters:
    -----------
    weekly_data : Dict[str, pd.DataFrame]
        Dictionary containing processed data
        for each week of data after having been processed by the
        `process_sales_batch` and `combine_sales_with_asset_data` funcation.
        Keys: Week identifiers (e.g., '2023-01', '2023-02')
        Values: DataFrames with the processed and combined data for each week
        
    Returns:
    --------
    tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]
        - weekly_comparison: Pivot table comparing metrics across weeks
        - summary: Overall summary across all weeks
        - all_data: Complete consolidated dataset
    """  
    all_weeks_data = pd.concat(combined_weekly_data.values(), ignore_index=True)
    
    weekly_comparison = pd.pivot_table(
        all_weeks_data, 
        values=['MWh', 'sales_amount', 'price', 'revenue_per_mwh'],
        index=['portfolio_id', 'asset_type'],
        columns='year_week'
    )
    
    weekly_comparison = weekly_comparison.reset_index()
    
    summary = all_weeks_data.groupby(['portfolio_id', 'asset_type']).agg({
        'MWh': 'sum',
        'sales_amount': 'sum',
        'transaction_count': 'sum',
        'price': 'mean',
        'asset_count': 'first',
        'primary_geography': 'first',
        'portfolio_age_years': 'first'
    }).reset_index()
    
    summary['avg_weekly_revenue'] = summary['sales_amount'] / len(combined_weekly_data)
    summary['avg_revenue_per_mwh'] = summary['sales_amount'] / summary['MWh']
    
    return weekly_comparison, summary, all_weeks_data

-------------------------
### Examples Outputs

In [ ]:
sales_data, asset_data = create_sample_data()
weekly_results = get_weekly_sales_data(sales_data)

In [ ]:
result = process_sales_batch(weekly_results['2023-01'])
result.head()

In [ ]:
combine_sales_and_asset_data = combine_sales_with_asset_data(result, asset_data)
combine_sales_and_asset_data.head()